# Lesson 2: Federated Fine-tuning

[INSTRUCTOR NOTES]
- In this second lesson you will learn how to perform federated finetuning of an LLM
- Just like in the previous notebook, you will finetune a 70M LLM. But will provide you with everything you need to finetune a larger 7B model.
- Recall that in the previous notebook you played the role of a scientist in a hospital that could only make use of the data available in the hospital. This prevented you from making use of all the data, limiting you to just 10% of the total data
- This time, you'll play the role of a scientist running the finetuning in a federated manner. This will allow you to train the LLM on 100% of the data **without having to move the data**
- In this notebook you'll make use of Flower's Simulation Engine to simulate a federation of 20 nodes (or clients), each representing a hosptial.
- **A good point to remind students about C1 that introduces step-by-step FL with Flower**

[INSTRUCTOR NOTES]
- Let's begin by importing a few packages and utility functions
- This notebook also makes use of HuggingFace and PEFT
- Let me highlight some imports that are of particular relevance for this notebook
    - Let's import Flower (`flwr`) which 
    - You'll make use of `flower-datasets` to download and partition the FedAlapaca dataset into 20 disjoint datasets (one for each hostpital remember?)
    - This example makes use of DP, so we need to import a mod (enables a flower client to use DP) and a wrapper for a Flower strategy to apply DP after aggregation.
    - Here you also import a few utility functions that help keeping the code in this notebook shorter

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import flwr as fl
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner
from datasets import load_dataset
from flwr.client.mod import fixedclipping_mod
from flwr.server.strategy import (
    DifferentialPrivacyClientSideFixedClipping
)
from utils.utils import * 
from utils.LLM import LLM_fl

Just like in the first notebook of this course, you will be interacting with two LLMs: a 70M LLM that is small enough to run efficiently in this notebook (you will learn how to finetune it via Federated Learning using Flower!); and a larger 7B LLM that has been finetuned with Flower and that you'll have the chance to query through the fireworks.ai API we provide you with. Later in the notebook, we give you all the tools to finetune the 7B model on your own. Please note that using config files with Flower is entirely optional.

## Federated LLM Finetuning with Differential Privacy

Let's begin by loading a config file that contains all the settings and training hyperparameters for this notebook. You will notice the sections in the config about the `dataset`, the `model` and the training settings (under the `train` section) are almost identical. This is because you will be finetuning the same [70M LLM](https://huggingface.co/EleutherAI/pythia-70m) as before and using the same [Medical Q&A](https://huggingface.co/datasets/medalpaca/medical_meadow_medical_flashcards) dataset.

### Load config

[INSTRUCTOR NOTES]
 - Let's begin by loading the config
 - As you can see its content is very similar to the one used in the centralized finetuning notebook
    - It uses the MedAlpaca dataset
    - It uses EleutherAI's model
    - It uses similar training settings

In [ ]:
# Load config
cfg = get_config("federated")

# Inspect the config
print_config(cfg)

Note while the majority of the config file closely matches the one used for centralized finetuning in the previous notebook of this course, a new section in the config under the `flower` tag has been added. This block of settings helps us define the Federated Finetuning process. Let's take a closer look:

[INSTRUCTOR NOTES]
- However there is a new section in the config under the `flower` tag (note you could have named it in a different way like "FL")
- This section contains arguments that will configure the FL finetuning process:
    - it specifies the total number of clients: 20
    - how many rounds of FL to perform: this minimal config just does 2
    - it involves 20% of the clients in each round. This is represented by the `fraction_fit` setting
    - Then with `client_resources` we can adjust the degree of parallelism of the simulation. This represent the resources a client needs, intuitively if you lower them, more clients will be able to run concurrently under the same hardware. For the 70M LLM seting 2 CPU cores per client is sufficient for this demonstration. However, you might want to increase it or even use a GPU when finetuning via FL larger models.
    - finally, the DP section specifies the `clipping_norm` and `noise_multiplier` parameters. These will add a some protection during the finetuning process, preventing data leakage.

In [ ]:
# Let's print the "flower" block of the config for easy inspection
print_config(cfg.flower)

The config includes the following parameters:
- `num_clients`: this refers to the total number of clients involved in the federation. You can think of each client being a different hospital or medical institution. They all want to collaboratively train a single model (an LLM for Medical Q&A in this case) but without sharing their data.
- `num_rounds`: Determines the number of FL rounds. This is one of the key hyperparameters in FL, depending on the application and complexity of the model/data, this could sometimes be in the order of hundreds or even a few thousands. In this notebook, you will use two rounds (but you are welcome to change this)
- `fraction_fit`: In Flower, the _strategies_ (e.g. `FedAvg` or any other of the many strategies [implemented in Flower](https://flower.ai/docs/framework/ref-api/flwr.server.strategy.html#module-flwr.server.strategy)) will sample a number of the available clients. With this parameter, you can configure the ratio of clients to be sampled. In this case is `0.2` which means 20% of the connected clients (i.e. 20) will be sampled in each round. This means that each round `4` clients will finetune the LLM on their local data.
- `client_resources`: You will learn how to run a FL simulation using Flower's Simulation Engine. The values in `client_resources` allow you to control the degree of parallelism for your simulation (i.e. how many clients can run concurrently). For this notebook, each client will use 2x CPU threads and no GPU. You can read more about [Flower Simulations](https://flower.ai/docs/framework/how-to-run-simulations.html) in the documentation.
- `dp`: This block of the config allows you to parameterize the Differentiable Privacy (DP) component used to finetune the LLM. There are different variations of DP, and in this notebook, you will use DP with fixed clipping as first proposed by [_McMahan et al._ (2018)](https://arxiv.org/abs/1710.06963). The higher these values are, the stronger the amount of noise added is. Higher noise values offer higher degrees of protection but at the expense of reduced model performance. This flavor of DP uses two main hyperparameters:
     - `noise_mult` (short for _noise multiplier_): which controls the amount of noise added to the weights after aggregation on the server side.
     - `clip_norm`: threshold value to clip the model updates at the client side and after aggregation.
 
In this notebook, you will add DP introducing a small amount of noise since we are prioritising model utility. You can read more about how to use DP in Flower in [the documentation](https://flower.ai/docs/framework/how-to-use-differential-privacy.html).

### Partition the dataset and prepare tokenizer

In this lesson, you will be simulating a FL setup involving 20 clients or nodes. Simulations often involve artificially splitting an existing dataset into as many disjoint sets as clients in the federation. Let's start by doing that. Let's split the Medical Q&A dataset into 20 partitions.

To do this we rely on [Flower Dataset](https://flower.ai/docs/datasets/) a library that greatly eases the download, partitioning and preprocessing of datasets for federated learning. Through Flower Dataset you have access to the whole library of 🤗Hugging Face Dataset (yes, over 100k datasets!). Let's keep things simple and split the dataset uniformly (in the FL jargon we often refer to this as IID partitioning)

[INSTRUCTOR NOTES]
- Let's beging by partitioning the MedAlpaca dataset into 20 disjoint sets. Each will become the "local dataset" of each of the 20 clients in the federation we are about to simulate. Recall each client here represents a hospital.
- Here we make use of Flower Datasets, a toolkit to download, process and partition datasets. Dataset partitioning can get fairly complex, specially when you want to introduce different forms of data heterogeneity into the mix. With Flower Dataset (which comes with several partitioners built-in) you can forget about all this and put all your attention into the design of your algorithm.
- Here we take the simplest approach and split the dataset with an IID partitioner, meaning that all partitions will be constructed by uniformly sampling from the whole dataset.
- Let's run the code and inspec the metadata of the first partition.
- [ run the code]
- Just like in the previous notebook, the dataset contains two columns `instruction` and `response`.

In [ ]:
# Generate data partitions using Flower Datasets
# Each client will make use of a different data partition
partitioner = IidPartitioner(num_partitions=cfg.flower.num_clients)
fds = FederatedDataset(
    dataset=cfg.dataset.name,
    partitioners={"train": partitioner}
)

# We can load one partition like this, let's print it's metadata
# you'll see it has a similar strucuture as what was shown in the
# notebook where you finetuned an LLM centrally
partition_zero = fds.load_partition(0) # load 0-th partition

# Format and print metadata
format_dataset(partition_zero)

Let's inspect the partitions generated and also construct a bar plot showing how many training examples are assigned to each client. We use a simple partitioning mechanism, which is why all partitions have roughly the same number of training examples. You can design more complex setups by using non-IID partitioners (there are [several popular ones built in into Flower Datasets](https://flower.ai/docs/datasets/ref-api/flwr_datasets.partitioner.html))

[INSTRUCTOR NOTES]
- Let's do a simple visualization of each data parition and display the amount of training example each one has
- As you can see, all 20 partition has roungly 1700 training examples

In [ ]:
visualize_partitions(fds)

Let's also construct our tokenizer and other utility objects needed to train the LLM model. Note there is no difference here w.r.t what was done in the centralized setting. 

[INSTRUCTOR NOTES]
- Now that the dataset is ready, let's proceed with the rest of the components needed to run our federation
- Just like in the previous notebook, let's load the tokenizer and other components needed to pre-process the input to the LLM

In [ ]:
# Construct the tokenizer and other components
(
tokenizer,
data_collator,
formatting_prompts_func,
) = get_tokenizer_and_data_collator_and_propt_formatting(
    cfg.model.name, 
    cfg.train.padding_side,
)

## Create ClientApp  (with DP support)

A `ClientApp` defines the job each FL client will perform. In this lesson, the client will finetune the model specified in the config using PEFT on its local data. Once the client finishes this step, it will communicate the updated parameters back to the server for aggregation. Note that we are passing the model config (i.e. `cfg.model`) to the client. Internally, it will instantiate it using PEFT and only finetune locally a small fraction of the total LLM parameters.

Flower clients can make use of `mods` (short for _modifiers_) to augment their capabilities. There are several [mods built in into Flower](https://flower.ai/docs/framework/how-to-use-built-in-mods.html), you can define your own too. In this lesson, we will make use of the [`fixedclipping-mod`](https://github.com/adap/flower/blob/main/src/py/flwr/client/mod/centraldp_mods.py#L34) that ensures models that the client is going to send back to the server are clipped by value.

[INSTRUCTOR NOTES]
- Clients in Flower are defined via a `ClientApp`.
- The `ClientApp` can be constructed by specifying a `client_fn` (client function) that returns a Client object which knows how to: (1) instantiate a mdoel; and (2) run a training loop.
- The client will essentailly run a function just like the finetuning function you wrote in the first notebook.
- Finally, because this client is going to make use of DP, a Fix-clipping DP mod is passed. You can read more about DP and the use of mods in the flower documentation on flower.ai

In [ ]:
save_path = "./my_fl_model"
client = fl.client.ClientApp(
    client_fn=gen_client_fn(
        fds,
        tokenizer,
        formatting_prompts_func,
        data_collator,
        cfg.model, # pass model config
        cfg.train, # pass train config
        save_path,
    ),
    mods=[fixedclipping_mod] # We pass a "mod" to enable client-side DP
)

## Create ServerApp (with DP support)

In Flower, `ServerApp` can make use of _strategies_, which are a [family of Python classes that orchestrate the federated learning process](https://flower.ai/docs/framework/ref-api/flwr.server.strategy.html#module-flwr.server.strategy). They define stages such as client sampling; communicating the _global model_ (i.e. the model being federated, or a subset of it -- as it is the case in this lesson where we finetune the model) to the clients participating in a given round; receiving the models sent back by the clients; aggregate the model updates to derive a new _global model_; perform "central" evaluation of the _global model_; and more. 

In this lesson, you will make use of `FedAvg`, from [McMahan et al. (2017)](https://arxiv.org/abs/1602.05629). Despite its simplicity, this strategy is a good starting point for most scenarios and therefore we use it in this lesson. Many [more strategies](https://flower.ai/docs/framework/ref-api/flwr.server.strategy.html#module-flwr.server.strategy) are available in Flower. Because strategies play such a central role in FL, they offer a high degree of customization. See [the documentation](https://flower.ai/docs/framework/ref-api/flwr.server.strategy.FedAvg.html#flwr.server.strategy.FedAvg) to learn more about how to customize this strategy or how to create one of your own.

[INSTRUCTOR NOTES]
- With the `ClientApp` defined, let's turn our attention to the server side.
- At the core of a `ServerApp` there is a _strategy_. 
- A Flower _strategy_ is in charged of: sampling clients, communicating instructions to the clients, receiving model updates from the clients, run the aggregation of models, and more.
- In this notebook, you'll make use of FedAvg, a relateively simple but surprisingly effective method of aggregating model updates.
- [ copy code and highlight a cuple of arguments, most of them come from the config]
- [ remind students to check C1?]

In [ ]:
# Instantiate strategy according to config.
strategy = fl.server.strategy.FedAvg(
    min_available_clients=cfg.flower.num_clients, # total clients
    fraction_fit=cfg.flower.fraction_fit, # ratio of clients to sample
    fraction_evaluate=0.0, # No federated evaluation
    # A (optional) function used to configure a "fit()" round
    on_fit_config_fn=get_on_fit_config(),
    # A (optional) function to aggregate metrics sent by clients
    fit_metrics_aggregation_fn=fit_weighted_average,
    # A (optional) function to execute on the server after each round. 
    # In this example the function only saves the global model.
    evaluate_fn=get_evaluate_fn( 
        cfg.model,
        cfg.train.save_every_round,
        cfg.flower.num_rounds,
        save_path
    ),
)

In this lesson, because we want to make use of DP, the `ServerApp` needs to pass the _strategy_ object into a wrapper class that will apply fixed-clipping DP to the aggregated model. You can learn more about the wrapper [DifferentialPrivacyClientSideFixedClipping](https://flower.ai/docs/framework/ref-api/flwr.server.strategy.DifferentialPrivacyClientSideFixedClipping.html) class in the documentation.

[INSTRUCTOR NOTES]
- This lesson want's to "enhance" FL with DP
- To do that we use a "wrapping strategy" that will maintaing the behaviour of the FedAvg strategy you just created but will add extra functionality
- This extra functionality means in this case adding calibrated noise to the resulting aggregated model

In [ ]:
# Make FedAvg DP-ready with a wrapper class
sampled_clients = cfg.flower.num_clients*strategy.fraction_fit
strategy = DifferentialPrivacyClientSideFixedClipping(
    strategy, 
    noise_multiplier=cfg.flower.dp.noise_mult,
    clipping_norm=cfg.flower.dp.clip_norm, 
    num_sampled_clients=sampled_clients
)

Now that the strategy is ready, let's use it to instantiate the `ServerApp` object.

[INSTRUCTOR NOTES]
- Now that the _strategy_ is ready and includes DP, you can instantiate a `ServerApp` by also indicating the number of rounds to run the simulation for
- By default, the simluation will run for 2 rounds, but you are welcome to increate this number

In [ ]:
# ServerApp definition takes a Flower strategy
# and a config specifying the number of rounds.
num_rounds = cfg.flower.num_rounds
server = fl.server.ServerApp(
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
)

### Lauch Flower Simulation

Now that you have both `ClientApp` and `ServerApp` defined, you are ready to launch your FL simulation. With the configuration loaded at the beginning of this notebook, this FL simulation will run for 2 rounds, each involving 4 clients. Each client will perform just a few steps of local LLM finetuning. Here you'll be using the smaller 70M LLM.

[INSTRUCTOR NOTES]
- Now that the dataset, the `ClientApp` and the `ServerApp` are ready, you can launch the simulation
- To do this, you just need to run the `run_simulation` function
- In these notebooks, running the simulation should take a couple of minutes (after all you are finetuning with PEFT a small 70M LLM for a couple of rounds, each involving 4 clients)
- [run the code]

In [ ]:
# Run code showing oh FL finetuning works
client_resources = dict(cfg.flower.client_resources)
fl.simulation.run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=cfg.flower.num_clients,
    backend_config={"client_resources": client_resources}
)

[INSTRUCTOR NOTES] -- result of running the simulation above
- You can see the log is divided into 4 sections:
    - [INIT]: logging information about the initialization process.
    - [ROUND 1 and ROUND 2]: logging information about each round of FL.
        - note the lines `configure_fit sampled 4/20`; then `aggregate_fit received 4 results and 0 failures`
    - [SUMMARY]: where a summary of the simulation process is shown.
        - of interest is the last line, showing the average training loss at each client. 
        - note that in this example there is not "centralized evaluation" happening and this is why you see the loss being 0.0

### Finetune a 7B LLM with Flower on your own

[INSTRUCTOR NOTES]
- Now that you have done FL finetuning of a 70M LLM, let's see how you can run the finetuning of a model 100x larger (i.e. a Mistral 7B)
- It's actually very easy! you just need to replace the config file and run the same cells as above.
- Replace the cell at the top of the notebook with `cfg = get_config("federated_full")` <- note the `federated_full` instead of `federated`.

In [ ]:
# You can launch the full FL finetuning 
# using a larger Mistral-7B by running
# the same code as above but loading config 
# `federated_full.yaml`at the top of this notebook:
#
# cfg = get_config("federated_full")
#
# !! If you decide to do that, it's recommended running 
# the finetuning on a machine with one (or more) GPUs.
# Precisely these steps were used to train the FL model
# you'll make use of through fireworks.ai in the next section.

### Evaluating FL-finetuned model

Let's now evaluate the 7B LLM that results after completing FL finetuning. You can observe that the finetuned model very good job at Medical Q&A. 

[INSTRUCTOR NOTES]
- The result of running the `federated_full` config (which runs the larger 7B model and for 200 rounds) is what you'll have now the chance to interact with via the fireworks.ai API
- Let's evaluate the finetuned model via FL. 
- The code below should look very familiar, it's near identical to the one used in the first notebook
- [ run the code]
- You can observe that the response the LLM gave use is perfect!

In [ ]:
# Create an LLM evaluator using a Mistral-7B finetuned w/ Flower
llm_eval = LLM_fl()

# Let's load the whole dataset and extract the first example
train_dataset = load_dataset(cfg.dataset.name, split='train')
train_dataset = format_dataset(train_dataset)
example_index = 12

# Extract the an example
data_point = train_dataset[example_index]

# Pass Medical Q&A question
llm_eval.eval(data_point['instruction'], verbose=True)

# Fetch and print response
llm_eval.print_response()

# Expected response
ex_response = format_string(data_point['response'])
print(f"Expected output:\n\t{ex_response}")

### Assessing quality of Federated Finetuned model

The code cell below allows you to evaluate the performance of your finetuned LLM on a held-out dataset. Just as you did in the previous notebook, here you'll make use of the [PubMedQA](https://huggingface.co/datasets/bigbio/pubmed_qa) dataset.

[INSTRUCTOR NOTES]
- Now that you have finetuned the LLM with Flower, you migh want to evaluate it on the PubMed Q&A dataset.
- The steps for doing that are no different from the ones you saw in the first notebook (when you evaluated a centrally finetuned model)
- [copy the code, but don't run it]

In [ ]:
# Lauch evalution code
# from utils.utils import inference, evaluate

# Step 1: generate answers
# To run inference on the federated finetuned model:
# inference(base_model_name_path=cfg.model.name, 
#           peft_path=/path/to/fl_peft_checkpoint/, 
#           run_name="federated_finetuned",
#          )


# Step 2: evaluation --- accuracy value will be printed
# evaluate(run_name="federated_finetuned")

Let's compare the performance obtained by the three models you have interacted with so far. First, the original pre-trained model (i.e. the 7B-Mistral model that you have later on finetuned); a centrally-finetuned model that makes use of 10% of the total dataset; and finally the finetuned model through Federated Learning using Flower. We can clearly observer that, because FL doesn't require transferring the data to a central location, its potential to "make sense" out of more _distributed_ data leads to a far superior model.

[INSTRUCTOR NOTES]
- Let's see where does the FL-finetuned model stand compared to the pre-trained model and it's centrally-finetuned counterpart
- The difference is quit big (nearly 20% more accurate in FL)

In [ ]:
# Compare performance between 
# pre-trained, centralised and federated models
visualize_results(
    results=['7b/pretrained', '7b/cen_10', '7b/fl'])

### Normalize training setups

In the first lesson, you learned how to finetune a pre-trained model on a Medical Q&A dataset. We used 10% of the data to present the setup where having all data in a centralized location is unrealistic (especially when dealing with private or highly sensible data), as is the case with medical data. Still, we want to show what the performance of that model centrally finetuned on 100% would be.

[INSTRUCTOR NOTES]
- One could argue that under very concrete setups, moving all the data to a centralized location is an option.
- Assuming this is true (ignoring all the challenges associated with making it happen), we could repeat the centralized finetuning we saw in the first notebook.
- This results in a model performing very similarly to the one you can obtain via FL with DP.
- [INGORE THE CELL COMMENTED BELOW -- run the next one with the `visualize_results` code]

In [ ]:
# Repeat the steps in Lesson 1 but load the 
# whole dataset (instead of 10% of the data)
# train_dataset = load_dataset(cfg.dataset.name, split='train')
# Then run the code as in Lesson 1 but using the config file
# indicated for finetuning the 7B LLM model.

With the same amount of data at their disposal, both centralized and federated finetuning lead to an LLM of comparable quality. However, for a real-world centralized setup it seems unlikely that having access to 100% of the data would be possible due to privacy concerns, regulations, or logistical reasons.

In [ ]:
# Plot comparing approaches
visualize_results(
    results=['7b/pretrained', '7b/cen_10',
             '7b/cen_full', '7b/fl'],
    compact=True)

### What are the overheads of Federated Finetuning?

Unlike traditional centralized training, federated learning does not require transferring data to a centralized location. However, most forms of FL involve communicating model updates. What these _"updates"_ are vary depending on the FL method and the application but in the majority of settings they mean the whole or part of the model weights or gradients. It is therefore important to correctly dimensionate what are the _communication costs_ associated with a given FL application. 

In this lesson, we saw how to finetune an LLM. With PEFT (and LoRA under the hood) only a small fraction of the model parameters are updated on the client side, therefore only those need to be communicated between the server and the clients. We have put together a utility function to characterize the communication costs of the 70M and 7B config files mentioned in this notebook. The take-home message is that LLM finetuning with FL is feasible even for large LLMs. 

[INSTRUCTOR NOTES]
- Before concluding this notebook, let's analyse what are the communication costs associated with the 70M LLM you finetuned in this notebook.
- [ run the code]
- As you can see, thanks to PEFT the communication costs are reduced by over 300x
    - if you were to communicate the whole model, the finetuning you just did in this notebook would have communicated over 4GB of data
    - but because you used PEFT under-the-hood, your clients and server only exchanged 12M of data. Pretty good huh?
- Below you can see the output that using the larger (`federated_full`) config for the 7B model would output.
    - the communication savings are even larger, over three orders of magnitude!
    - This much longer experiment (runs for 200 rounds) exchanges a total of 41 GB of data -- for reference the FULL model is 27GB alone.
    - If we weren't using PEFT, the total exchange would have been of over 43 TeraBytes

In [ ]:
cfg = get_config("federated")
# Compute communication costs and estimate transmission
# time given a specific Mbps bandwidth
compute_communication_costs(cfg, comm_bw_mbps=20)

If you recompute the communication costs analysis for the full 7B model (i.e. loading `federated_full`) you will see the following report. Note that even when finetuning with PEFT a 7B LLM, the number of parameters that need to be finetuned (and therefore shared between server-client) occupies just 26 MB at FP32 (i.e. 32-bit floating -oint precision):

```
Full model:
	7248.548 M parameters
	27651.02 MB --> upload in 11060.41s @ 20Mbps
Finetuned model:
	6.816 M parameters
	26.00 MB --> upload in 10.40s @ 20Mbps
Federated Learning setting: 
	Number of rounds: 200
	Number of clients per round: 4
-----------------------------------------------
Communication costs (Full model): 43204.7 GB
Communication costs (Finetuning): 41600.0 MB
Communication savings: 1063.5x
```